In [397]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
##### https://github.com/arpanghosh8453/programs/blob/master/Fitbit%20Data%20Analyzer/Fitbit%20HR%20analyzer.ipynb
# https://dev.fitbit.com/
# https://dev.fitbit.com/build/reference/web-api/basics/

# You need the following modules, if you don't have them, use pip install <module-name>
import requests
import pandas as pd
import datetime
import numpy as np
import json

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import jupyter_dash
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Notes

In [2]:
notes_df = pd.read_csv('data/notes.csv')
notes_df

,Timestamp,Have an idea? Need to remember something?
0,9/30/2020 23:35:35,Need fleece jacket
1,10/1/2020 6:22:07,Add a column of percent of time spent on activ...
2,10/1/2020 6:26:56,"Add column, percent of time at work spent not ..."
3,10/1/2020 8:16:23,"Add column time since last sleep, clean, etc. ..."
4,10/3/2020 8:01:31,Make a generic human chart(t pose) that shows ...
...,...,...
138,6/23/2021 14:25:05,Tanning by the pool. edit: got moderately burn...
139,6/25/2021 5:03:15,Lots of water in my apartment. Raining outside
140,6/25/2021 9:44:57,Mom came over to help me clean
141,7/4/2021 4:19:16,On blog site have personal datasets viewable/d...


In [3]:
df = notes_df

In [4]:
df["Day"] = pd.to_datetime(df['Timestamp']).dt.date
df

,Timestamp,Have an idea? Need to remember something?,Day
0,9/30/2020 23:35:35,Need fleece jacket,2020-09-30
1,10/1/2020 6:22:07,Add a column of percent of time spent on activ...,2020-10-01
2,10/1/2020 6:26:56,"Add column, percent of time at work spent not ...",2020-10-01
3,10/1/2020 8:16:23,"Add column time since last sleep, clean, etc. ...",2020-10-01
4,10/3/2020 8:01:31,Make a generic human chart(t pose) that shows ...,2020-10-03
...,...,...,...
138,6/23/2021 14:25:05,Tanning by the pool. edit: got moderately burn...,2021-06-23
139,6/25/2021 5:03:15,Lots of water in my apartment. Raining outside,2021-06-25
140,6/25/2021 9:44:57,Mom came over to help me clean,2021-06-25
141,7/4/2021 4:19:16,On blog site have personal datasets viewable/d...,2021-07-04


In [5]:
# # https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/09_timeseries.html
# grouped_notes = average_notes.groupby([pd.to_datetime(df["Day"]).dt.date])
# grouped_notes = pd.DataFrame(grouped_notes)
# grouped_notes

# sum_of_notes_per_day = []
# for note in grouped_notes[1]:
#     sum_of_notes_per_day.append(len(note))
# sum_of_notes_per_day

In [6]:
by_day = pd.to_datetime(notes_df['Day']).dt.to_period('D').value_counts().sort_index()
by_day.index = pd.PeriodIndex(by_day.index)
df_day = by_day.rename_axis('Day').reset_index(name='Counts')
df_day

,Day,Counts
0,2020-09-30,1
1,2020-10-01,3
2,2020-10-03,1
3,2020-10-07,1
4,2020-10-10,2
...,...,...
92,2021-06-18,1
93,2021-06-20,2
94,2021-06-23,2
95,2021-06-25,2


In [7]:
df['Counts'] = 0
df.head(5)

,Timestamp,Have an idea? Need to remember something?,Day,Counts
0,9/30/2020 23:35:35,Need fleece jacket,2020-09-30,0
1,10/1/2020 6:22:07,Add a column of percent of time spent on activ...,2020-10-01,0
2,10/1/2020 6:26:56,"Add column, percent of time at work spent not ...",2020-10-01,0
3,10/1/2020 8:16:23,"Add column time since last sleep, clean, etc. ...",2020-10-01,0
4,10/3/2020 8:01:31,Make a generic human chart(t pose) that shows ...,2020-10-03,0


In [8]:
df_day['Day'] = df_day['Day'].astype(str)
df_day['Day'] = pd.to_datetime(df_day['Day']).dt.date
print(type(df_day['Day'][0]))
print(type(df['Day'][0]))
print()
print(df.dtypes)
print()
print(df_day.dtypes)

<class 'datetime.date'>
<class 'datetime.date'>

Timestamp                                    object
Have an idea? Need to remember something?    object
Day                                          object
Counts                                        int64
dtype: object

Day       object
Counts     int64
dtype: object


In [9]:
df_day['Counts'].max()

10

In [10]:
df_day['Counts'].mean()

1.4742268041237114

In [11]:
df_day['Counts'].median()

1.0

In [12]:
df_day['Counts'].std()

1.0713385660117232

In [13]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html
df = pd.merge(df, df_day, how='right', on='Day')
df

,Timestamp,Have an idea? Need to remember something?,Day,Counts_x,Counts_y
0,9/30/2020 23:35:35,Need fleece jacket,2020-09-30,0,1
1,10/1/2020 6:22:07,Add a column of percent of time spent on activ...,2020-10-01,0,3
2,10/1/2020 6:26:56,"Add column, percent of time at work spent not ...",2020-10-01,0,3
3,10/1/2020 8:16:23,"Add column time since last sleep, clean, etc. ...",2020-10-01,0,3
4,10/3/2020 8:01:31,Make a generic human chart(t pose) that shows ...,2020-10-03,0,1
...,...,...,...,...,...
138,6/23/2021 14:25:05,Tanning by the pool. edit: got moderately burn...,2021-06-23,0,2
139,6/25/2021 5:03:15,Lots of water in my apartment. Raining outside,2021-06-25,0,2
140,6/25/2021 9:44:57,Mom came over to help me clean,2021-06-25,0,2
141,7/4/2021 4:19:16,On blog site have personal datasets viewable/d...,2021-07-04,0,2


In [14]:
# a1, a2 = df.align(df_day, join='left', axis=1)
# a1

In [15]:
# a2

In [16]:
# https://stackoverflow.com/questions/62672427/how-do-i-convert-iso-8601-object-into-datetime-format-in-a-pandas-dataframe
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.head(5)

,Timestamp,Have an idea? Need to remember something?,Day,Counts_x,Counts_y
0,2020-09-30 23:35:35,Need fleece jacket,2020-09-30,0,1
1,2020-10-01 06:22:07,Add a column of percent of time spent on activ...,2020-10-01,0,3
2,2020-10-01 06:26:56,"Add column, percent of time at work spent not ...",2020-10-01,0,3
3,2020-10-01 08:16:23,"Add column time since last sleep, clean, etc. ...",2020-10-01,0,3
4,2020-10-03 08:01:31,Make a generic human chart(t pose) that shows ...,2020-10-03,0,1


In [17]:
df["Timestamp"] = df["Timestamp"].apply(lambda x: x.replace(year=1970, month=1, day=1))
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df

,Timestamp,Have an idea? Need to remember something?,Day,Counts_x,Counts_y
0,1970-01-01 23:35:35,Need fleece jacket,2020-09-30,0,1
1,1970-01-01 06:22:07,Add a column of percent of time spent on activ...,2020-10-01,0,3
2,1970-01-01 06:26:56,"Add column, percent of time at work spent not ...",2020-10-01,0,3
3,1970-01-01 08:16:23,"Add column time since last sleep, clean, etc. ...",2020-10-01,0,3
4,1970-01-01 08:01:31,Make a generic human chart(t pose) that shows ...,2020-10-03,0,1
...,...,...,...,...,...
138,1970-01-01 14:25:05,Tanning by the pool. edit: got moderately burn...,2021-06-23,0,2
139,1970-01-01 05:03:15,Lots of water in my apartment. Raining outside,2021-06-25,0,2
140,1970-01-01 09:44:57,Mom came over to help me clean,2021-06-25,0,2
141,1970-01-01 04:19:16,On blog site have personal datasets viewable/d...,2021-07-04,0,2


In [18]:
# https://stackoverflow.com/questions/52007139/get-the-average-date-from-multiple-dates-pandas
average_notes = pd.DataFrame()
average_notes['Timestamp'] = [x.value for x in df['Timestamp']]
print(average_notes.head(5))
average_notes.describe()

        Timestamp
0  84935000000000
1  22927000000000
2  23216000000000
3  29783000000000
4  28891000000000


,Timestamp
count,1.430000e+02
mean,3.670466e+13
std,2.418635e+13
min,2.870000e+11
25%,1.689150e+13
50%,3.157600e+13
75%,5.806100e+13
max,8.624400e+13


In [19]:
# https://stackoverflow.com/questions/19231871/convert-unix-time-to-readable-date-in-pandas-dataframe
# https://stackoverflow.com/questions/54733465/pandas-how-to-extract-hhmm-from-datetime-column-in-python/54733547
# https://stackoverflow.com/questions/39662149/pandas-extract-date-and-time-from-timestamp

mean_notes = pd.to_datetime(average_notes['Timestamp'].mean())
mean_notes_time = mean_notes.strftime('%H:%M:%S')
print(f"Mean Notes: {mean_notes}")

median_notes = pd.to_datetime(average_notes['Timestamp'].median())
median_notes_time = median_notes.strftime('%H:%M:%S')
print(f"Median Notes: {median_notes}")

Mean Notes: 1970-01-01 10:11:44.664335664
Median Notes: 1970-01-01 08:46:16


In [20]:
rolling=7
rolling_timestamp = average_notes.rolling(rolling).mean()
rolling_timestamp['Timestamp'] = rolling_timestamp.apply(pd.to_datetime)
rolling_timestamp.head(10)

,Timestamp
0,NaT
1,NaT
2,NaT
3,NaT
4,NaT
5,NaT
6,1970-01-01 08:57:33.428571428
7,1970-01-01 08:18:22.714285714
8,1970-01-01 08:03:15.428571428
9,1970-01-01 07:34:10.714285714


In [21]:
print(type(rolling_timestamp['Timestamp'][38]))
rolling_timestamp['Timestamp'][38]

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


Timestamp('1970-01-01 07:04:52.571428571')

In [22]:
# for time in rolling_timestamp['Timestamp'][6:]:
#     time = time.strftime('%H:%M:%S')
# print(rolling_timestamp['Timestamp'].head(10))

In [23]:
df['Day'][0]

datetime.date(2020, 9, 30)

In [24]:
df['Day'] = df['Day'].apply(str)
df['Day']

0      2020-09-30
1      2020-10-01
2      2020-10-01
3      2020-10-01
4      2020-10-03
          ...    
138    2021-06-23
139    2021-06-25
140    2021-06-25
141    2021-07-04
142    2021-07-04
Name: Day, Length: 143, dtype: object

In [25]:
df

,Timestamp,Have an idea? Need to remember something?,Day,Counts_x,Counts_y
0,1970-01-01 23:35:35,Need fleece jacket,2020-09-30,0,1
1,1970-01-01 06:22:07,Add a column of percent of time spent on activ...,2020-10-01,0,3
2,1970-01-01 06:26:56,"Add column, percent of time at work spent not ...",2020-10-01,0,3
3,1970-01-01 08:16:23,"Add column time since last sleep, clean, etc. ...",2020-10-01,0,3
4,1970-01-01 08:01:31,Make a generic human chart(t pose) that shows ...,2020-10-03,0,1
...,...,...,...,...,...
138,1970-01-01 14:25:05,Tanning by the pool. edit: got moderately burn...,2021-06-23,0,2
139,1970-01-01 05:03:15,Lots of water in my apartment. Raining outside,2021-06-25,0,2
140,1970-01-01 09:44:57,Mom came over to help me clean,2021-06-25,0,2
141,1970-01-01 04:19:16,On blog site have personal datasets viewable/d...,2021-07-04,0,2


## Creating '\<br>' in notes

In [26]:
counts = []
notes = []
for note in df['Have an idea? Need to remember something?']:
    counts.append(len(note.split()))
    notes.append(note.split())
notes[:3]

[['Need', 'fleece', 'jacket'],
 ['Add',
  'a',
  'column',
  'of',
  'percent',
  'of',
  'time',
  'spent',
  'on',
  'activity',
  'in',
  'day/month/year',
  'after',
  'excluding',
  'sleep'],
 ['Add',
  'column,',
  'percent',
  'of',
  'time',
  'at',
  'work',
  'spent',
  'not',
  'working',
  'i.e.',
  'reading',
  'programming',
  'etc.']]

In [27]:
print(counts)

[3, 15, 14, 11, 20, 11, 6, 34, 13, 12, 22, 3, 18, 2, 6, 10, 12, 12, 31, 34, 10, 3, 6, 9, 21, 3, 11, 9, 5, 17, 36, 6, 20, 2, 12, 26, 11, 26, 46, 3, 10, 19, 18, 8, 4, 3, 42, 4, 29, 12, 15, 6, 10, 31, 22, 22, 19, 9, 52, 18, 4, 17, 11, 27, 8, 11, 3, 14, 26, 35, 19, 6, 35, 40, 20, 17, 24, 17, 27, 5, 4, 6, 4, 20, 11, 17, 13, 17, 6, 6, 12, 58, 30, 9, 29, 27, 7, 19, 30, 55, 8, 24, 28, 13, 9, 2, 4, 11, 4, 3, 19, 12, 13, 18, 7, 10, 6, 88, 2, 25, 5, 2, 10, 16, 7, 42, 10, 21, 25, 5, 88, 6, 16, 5, 1, 8, 1, 2, 15, 8, 7, 13, 3]


In [28]:
df['Word Count'] = counts
df

,Timestamp,Have an idea? Need to remember something?,Day,Counts_x,Counts_y,Word Count
0,1970-01-01 23:35:35,Need fleece jacket,2020-09-30,0,1,3
1,1970-01-01 06:22:07,Add a column of percent of time spent on activ...,2020-10-01,0,3,15
2,1970-01-01 06:26:56,"Add column, percent of time at work spent not ...",2020-10-01,0,3,14
3,1970-01-01 08:16:23,"Add column time since last sleep, clean, etc. ...",2020-10-01,0,3,11
4,1970-01-01 08:01:31,Make a generic human chart(t pose) that shows ...,2020-10-03,0,1,20
...,...,...,...,...,...,...
138,1970-01-01 14:25:05,Tanning by the pool. edit: got moderately burn...,2021-06-23,0,2,15
139,1970-01-01 05:03:15,Lots of water in my apartment. Raining outside,2021-06-25,0,2,8
140,1970-01-01 09:44:57,Mom came over to help me clean,2021-06-25,0,2,7
141,1970-01-01 04:19:16,On blog site have personal datasets viewable/d...,2021-07-04,0,2,13


In [29]:
# https://stackoverflow.com/questions/31040525/insert-element-in-python-list-after-every-nth-element
note2 = []
for note in notes:
    note = [x for y in (note[i:i+7] + ['<br>'] * (i < len(note) - 1) for i in range(0, len(note), 7)) for x in y]
    note2.append(note)
note2[:3]

[['Need', 'fleece', 'jacket', '<br>'],
 ['Add',
  'a',
  'column',
  'of',
  'percent',
  'of',
  'time',
  '<br>',
  'spent',
  'on',
  'activity',
  'in',
  'day/month/year',
  'after',
  'excluding',
  '<br>',
  'sleep'],
 ['Add',
  'column,',
  'percent',
  'of',
  'time',
  'at',
  'work',
  '<br>',
  'spent',
  'not',
  'working',
  'i.e.',
  'reading',
  'programming',
  'etc.',
  '<br>']]

In [30]:
s = ' '
note3 = []
for note in note2:
    note3.append(s.join(note))
    
note3[:3]

['Need fleece jacket <br>',
 'Add a column of percent of time <br> spent on activity in day/month/year after excluding <br> sleep',
 'Add column, percent of time at work <br> spent not working i.e. reading programming etc. <br>']

In [31]:
df['Have an idea? Need to remember something?'] = note3
df['Have an idea? Need to remember something?']

0                                Need fleece jacket <br>
1      Add a column of percent of time <br> spent on ...
2      Add column, percent of time at work <br> spent...
3      Add column time since last sleep, clean, <br> ...
4      Make a generic human chart(t pose) that <br> s...
                             ...                        
138    Tanning by the pool. edit: got moderately <br>...
139    Lots of water in my apartment. Raining <br> ou...
140                  Mom came over to help me clean <br>
141    On blog site have personal datasets viewable/d...
142    https://stackoverflow.com/questions/2560880/ho...
Name: Have an idea? Need to remember something?, Length: 143, dtype: object

In [32]:
df['Rolling Timestamp'] = rolling_timestamp
df

,Timestamp,Have an idea? Need to remember something?,Day,Counts_x,Counts_y,Word Count,Rolling Timestamp
0,1970-01-01 23:35:35,Need fleece jacket <br>,2020-09-30,0,1,3,NaT
1,1970-01-01 06:22:07,Add a column of percent of time <br> spent on ...,2020-10-01,0,3,15,NaT
2,1970-01-01 06:26:56,"Add column, percent of time at work <br> spent...",2020-10-01,0,3,14,NaT
3,1970-01-01 08:16:23,"Add column time since last sleep, clean, <br> ...",2020-10-01,0,3,11,NaT
4,1970-01-01 08:01:31,Make a generic human chart(t pose) that <br> s...,2020-10-03,0,1,20,NaT
...,...,...,...,...,...,...,...
138,1970-01-01 14:25:05,Tanning by the pool. edit: got moderately <br>...,2021-06-23,0,2,15,1970-01-01 10:35:33.000000000
139,1970-01-01 05:03:15,Lots of water in my apartment. Raining <br> ou...,2021-06-25,0,2,8,1970-01-01 09:41:48.285714285
140,1970-01-01 09:44:57,Mom came over to help me clean <br>,2021-06-25,0,2,7,1970-01-01 07:47:31.428571428
141,1970-01-01 04:19:16,On blog site have personal datasets viewable/d...,2021-07-04,0,2,13,1970-01-01 07:50:21.714285714


In [33]:
# can add other customdata to the stack, maybe number of words per note?
customdata = np.stack((df['Counts_y'], 
                       df['Have an idea? Need to remember something?'],
                       df['Word Count']),
                       axis=1)
print(customdata.shape)
customdata[-3:]

(143, 3)


array([[2, 'Mom came over to help me clean <br>', 7],
       [2,
        'On blog site have personal datasets viewable/downloadable <br> like kaggle, for others to use <br>',
        13],
       [2,
        'https://stackoverflow.com/questions/2560880/how-to-get-contents-of-amazon-shopping-cart https://stackoverflow.com/questions/58577720/amazon-api-to-get-my-cart-items-and-previous-orders https://www.octoparse.com/# <br>',
        3]], dtype=object)

# Dashboard

In [34]:
# https://community.plotly.com/t/how-do-i-change-the-x-axis-representation-of-time-on-the-px-timeline/44834/4
# https://stackoverflow.com/questions/67319505/how-to-plot-multiple-timelines-in-one-graph
# https://stackoverflow.com/questions/66905297/plotly-is-it-possible-to-add-a-second-y-axis-to-plotly-timeline

title = 'Notes Dashboard'

# rolling=7
# rolling_timestamp = df['Timestamp'].rolling(rolling).mean()

fig = go.Figure()
fig = px.timeline(df, x_start="Timestamp", x_end="Timestamp", y='Day', )

# https://plotly.com/python/marker-style/
fig.add_trace(go.Scatter(x=df['Timestamp'], 
                         y=df['Day'], 
                         name='Timestamp', 
                         mode='markers',
                         marker_color='#636EFA',
                         
                         # https://chart-studio.plotly.com/~empet/15366/customdata-for-a-few-plotly-chart-types/#/
                         # https://plotly.com/python/hover-text-and-formatting/
                         customdata = customdata, 
                         hovertemplate =
                         "Timestamp: %{x} %{y}<br>" +
                         "Sum of Notes for Day: %{customdata[0]}<br>" +
                         "Word Count of Note: %{customdata[2]}<br>" +
                         "%{customdata[1]}"
                         "<extra></extra>",
                        ))

fig.add_trace(go.Scatter(x=df['Rolling Timestamp'], 
                         y=df['Day'], 
                         name=f'{rolling} Day Rolling Timestamp',
                         customdata = customdata, 
                         hovertemplate =
                         "{rolling} Day Rolling Mean Timestamp: %{x}<br>" +
                         "<extra></extra>",
                        ))

fig.add_vline(x=mean_notes, line_dash="dash")
fig.add_vline(x=median_notes, line_dash="dash")

# https://github.com/d3/d3-time-format#locale_format change time format
fig.update_layout(xaxis=dict(
                      title='Timestamp', 
                      tickformat = '%H:%M:%S',
                      range=['1970-01-01 00:00:00','1970-01-01 23:59:59']
                  ),
                  yaxis=dict(
                      title='Day',
                      tickformat = '%B/%d/%Y',
                      dtick="M1",
                      #dtick=86400000*2
                   ))

fig.update_layout(
    #hovermode='x unified',
    #dragmode='pan',
    annotations=[
        #https://plotly.com/python/text-and-annotations/#adding-annotations-with-xref-and-yref-as-paper
        #https://community.plotly.com/t/how-to-add-a-text-area-with-custom-text-as-a-kind-of-legend-in-a-plotly-plot/24349/3
        go.layout.Annotation(
            text=f'Mean Notes: {mean_notes_time}<br>Median Notes: {median_notes_time}',
            align='left',
            showarrow=False,
            xref='paper',
            yref='paper',
            yanchor="top",
            y=.98,
            xanchor="right",
            x=1,)])
    
fig.update_layout(#hovermode='x unified',
                  #dragmode='pan',
                  title=dict(text=title, 
                             yanchor="top",
                             y=.98,
                             xanchor="left",
                             x=.05),
                 legend=dict(orientation="h",
                             yanchor="bottom",
                             y=1.02,
                             xanchor="right",
                             x=1))
fig.update_yaxes(showspikes=True)
fig.update_yaxes(autorange="reversed", # otherwise tasks are listed from the bottom up
                 showspikes=True)

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")

app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '98vh',
               #'width': '150vh',
              }
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=8060, debug=True)

Dash app running on http://127.0.0.1:8060/


# Histogram

In [385]:
title = "Notes Histogram"

fig = go.Figure()
fig.add_trace(go.Histogram(x=df['Counts_y'], 
                           name="Sum of Notes for Day",
                           hovertemplate =
                           "Sum of Notes for Day: %{x}<br>" +
                           "Number of Notes with that sum: %{y}<br>"
                           "<extra></extra>",))
fig.add_trace(go.Histogram(x=df['Word Count'], 
                           name="Number of Words in Note",
                           hovertemplate =
                           "Number of Words in Note: %{x}<br>" +
                           "Number of Notes with that Word Count: %{y}<br>"
                           "<extra></extra>",
                           marker_color='#00CC96',))

# Overlay both histograms
fig.update_layout(barmode='overlay')

# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75, 
                  # https://plotly.com/python/reference/histogram/#histogram-xbins
                  xbins_size=1,
                  #xbins_start=60000,
                  #xbins_end=86394000,
                 )

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")

app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '98vh',
               #'width': '150vh',
              }
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=9000, debug=True)

Dash app running on http://127.0.0.1:9000/
